# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('output_data/cities.csv')
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.5906,26.8910,53.44,47,1,10.29,ZA,1627177551
1,Butaritari,3.0707,172.7902,83.95,72,13,12.24,KI,1627177535
2,Hilo,19.7297,-155.0900,80.33,85,90,8.05,US,1627177580
3,Severo-Kuril'sk,50.6789,156.1250,57.61,89,98,2.15,RU,1627177581
4,Bengkulu,-3.8004,102.2655,81.73,67,1,1.21,ID,1627177581


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(g_key)
#store 'lat' and 'lng' into locations
locations = df[['Lat', 'Lng']]
#store humidity values
humidity = df['Humidity']

In [6]:
max_humidity = df['Humidity'].max().astype(float)

In [7]:
#creating humidity heatmap layer
fig = gmaps.figure(center = (20.0, 15.0), zoom_level = 2)

heatlayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, 
                                max_intensity = max_humidity, point_radius = 3)

fig.add_layer(heatlayer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.
*A max temperature lower than 80 degrees but higher than 70.
*Wind speed less than 10 mph.
*Zero cloudiness.
*Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [8]:
new_df = df.loc[(df['Max Temp']<80) & (df['Max Temp']>70) & (df['Wind Speed']<10) & 
               (df['Cloudiness']==0)].dropna()
new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,Ribeira Brava,32.6500,-17.0667,73.00,79,0,2.95,PT,1627177621
150,Nova Olímpia,-14.7972,-57.2881,74.80,44,0,4.36,BR,1627177633
259,Natal,-5.7950,-35.2094,75.42,73,0,9.22,BR,1627177431
272,Qurayyat,31.3318,37.3428,75.38,57,0,4.61,SA,1627177678
275,Ponta do Sol,32.6667,-17.1000,73.00,77,0,2.82,PT,1627177679
284,Jardim,-21.4803,-56.1381,70.48,44,0,3.56,BR,1627177682
304,Novoil'insk,51.6829,108.7143,73.83,44,0,2.24,RU,1627177690
305,Dossor,47.5272,52.9811,77.59,33,0,6.17,KZ,1627177690
306,Jalu,29.0331,21.5482,77.13,43,0,6.89,LY,1627177691
315,Corinth,37.9407,22.9573,79.90,49,0,2.82,GR,1627177694


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = new_df
hotel_df['Hotel Name']= ''

In [10]:
#inputing url for the certain google search
#making parameters for particular search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'type': 'hotel', 
          'keyword':'hotel', 
          'radius':5000, 
          'key':g_key}


In [11]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['location'] = f'{lat}, {lng}'
    
    response = requests.get(base_url, params = params).json()
    results = response['results']
    #print(results)
    
    try:
        hotel = results[0]['name']
        print(f'The closest hotel to you in {city} is {hotel}.')
        print(f'-------------------------------------------------------------------------')
    
    except:
        print(f'There is no hotel close to you within a 5000m radius in {city}' )
        print(f'-------------------------------------------------------------------------')

The closest hotel to you in Ribeira Brava is Estalagem Da Ponta Do Sol.
-------------------------------------------------------------------------
The closest hotel to you in Nova Olímpia is Hotel Pantanal.
-------------------------------------------------------------------------
The closest hotel to you in Natal is Wish Natal.
-------------------------------------------------------------------------
The closest hotel to you in Qurayyat is The life of the Knights Hotel Suites.
-------------------------------------------------------------------------
The closest hotel to you in Ponta do Sol is Hotel Enotel Baía.
-------------------------------------------------------------------------
The closest hotel to you in Jardim is Hotel Jardim.
-------------------------------------------------------------------------
There is no hotel close to you within a 5000m radius in Novoil'insk
-------------------------------------------------------------------------
The closest hotel to you in Dossor is Go

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))